In [1]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
from pathlib import Path
from dotenv import load_dotenv
from random import randint
import pandas as pd 
import re, os, time, sys, csv, json

from toolkit import ordergenerator as og
from toolkit import general_tools as gt


In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
# self.chrome_options.add_argument('--headless')
chrome_options.add_argument('--incognito')
driver = webdriver.Chrome(options=chrome_options)

In [3]:
url = "https://www.booking.com/searchresults.fr.html?label=gog235jc-1DCAYoTUIGc2F2b2llSA1YA2hNiAEBmAENuAEXyAEM2AED6AEB-AECiAIBqAIDuAK37uOQBsACAdICJDJjMjgxMmVhLTIyM2MtNDI1Mi1iYTM4LTA3MmE4MjI3MWFkMdgCBOACAQ&sid=086e078aef1832684e5d5671c99a12b1&aid=356980&dest_id=2557&dest_type=region&nflt=ht_id%3D204%3Brpt%3D1&shw_aparth=0&checkin=2024-10-16&checkout=2024-10-17&selected_currency=EUR&lang=frhttps://www.booking.com/searchresults.fr.html?label=gog235jc-1DCAYoTUIGc2F2b2llSA1YA2hNiAEBmAENuAEXyAEM2AED6AEB-AECiAIBqAIDuAK37uOQBsACAdICJDJjMjgxMmVhLTIyM2MtNDI1Mi1iYTM4LTA3MmE4MjI3MWFkMdgCBOACAQ&sid=086e078aef1832684e5d5671c99a12b1&aid=356980&dest_id=2557&dest_type=region&nflt=ht_id%3D204%3Brpt%3D1&shw_aparth=0&checkin=2024-10-16&checkout=2024-10-17&selected_currency=EUR&lang=fr"

In [4]:
driver.get(url)

In [6]:
page = driver.page_source

In [7]:
soupe = BeautifulSoup(page, 'lxml')
cards = soupe.find_all('div', {'data-testid':"property-card"})

In [117]:
(len(cards) // 5) * 5

165

In [118]:
card = cards[0]

In [119]:
nom = card.find('div', {'data-testid':'title'}).text
print(f'nom: {nom}')

nom: Le Grand Balcon Hotel


In [87]:
prix_init = card.find('div', {'data-testid':"availability-rate-information"}).find('div', {'tabindex':'0'}).find('span', {'class':'f018fa3636 d9315e4fb0'}).text[1:].replace(u'\xa0', u'')
print(f'prix init: {prix_init}')

prix init: 137


In [88]:
prix_actuel = card.find('div', {'data-testid':"availability-rate-information"}).find('span', {'data-testid':'price-and-discounted-price'}).text[1:].replace(u'\xa0', u'')
print(f"prix actuel: {prix_actuel}")

prix actuel: 96


In [89]:
taxe_text = card.find('div', {'data-testid':'taxes-and-charges'}).text.replace(u'\xa0', u'')
taxe_text =  0 if 'compri' in taxe_text else taxe_text.split('€')[1]
print(f"taxes: {taxe_text}")

taxes: 35


In [91]:
adress = card.find('span', {'data-testid':'address'}).text
print(f'adress: {adress}')

adress: Aix-en-Provence


In [182]:
typologie = cards[5].find('h4').text.replace(u'\xa0', ' ')
print(f'typologie: {typologie}')

typologie: Chambre Standard - 1 Lit Queen-Size


In [8]:
next_button = soupe.find('button', {'class':"a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e bb803d8689 a16ddf9c57"})
if next_button:
    next_page = driver.find_element(By.XPATH, "//button[@aria-label='Page suivante']")
    next_page.click()
    WebDriverWait(driver, 1)

In [9]:
btn_vew_more = driver.find_element(By.XPATH, '//*[@id="bodyconstraint-inner"]/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div/button')

In [10]:
btn_vew_more.location_once_scrolled_into_view
driver.execute_script("window.scrollBy(0, -250);")

In [37]:
driver.get("https://www.booking.com/searchresults.fr.html?aid=1610684&label=poitiers-jXH27xmgvrQtl*97lNnM9AS380966959568%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atiaud-2007851544456%3Akwd-354694588041%3Alp9056095%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YfqnDqqG8nt10AsofPfvtt0&lang=en-gb&sid=5515b0ac0d548c76157b31c274e5a55d&sb=1&sb_lp=1&src=city&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fcity%2Ffr%2Fpoitiers.fr.html%3Faid%3D1610684%26label%3Dpoitiers-jXH27xmgvrQtl%252A97lNnM9AS380966959568%253Apl%253Ata%253Ap1%253Ap2%253Aac%253Aap%253Aneg%253Afi%253Atiaud-2007851544456%253Akwd-354694588041%253Alp9056095%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9YfqnDqqG8nt10AsofPfvtt0%26sid%3D5515b0ac0d548c76157b31c274e5a55d%26inac%3D0%26%26&ss=Poitiers&is_ski_area=&ssne=Poitiers&ssne_untouched=Poitiers&city=-1458949&checkin_year=2024&checkin_month=09&checkin_monthday=28&checkout_year=2024&checkout_month=09&checkout_monthday=29&flex_window=0&efdco=1&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&dest_id=-&dest_type=city&lang=fr")

In [180]:
int(''.join(list(filter(str.isdigit, driver.find_element(By.XPATH, "//h1[@aria-live='assertive']").text))))

89

In [18]:
driver.find_element(By.XPATH, '//span[contains(text(), "Afficher plus de résultats")]').click()

In [11]:
def load_page_content():
    def scroll_page():
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    scroll_page()

    while True:
        try:
            scroll_page()
            btn_vew_more = driver.find_element(By.XPATH, '//*[@id="bodyconstraint-inner"]/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div/button')
            if btn_vew_more:
                driver.execute_script('arguments[0].scrollIntoView({ behavior: "smooth", block: "center", inline: "center" })', btn_vew_more)
                time.sleep(.5)
                btn_vew_more.click()
        except NoSuchElementException:
            break

    soupe = BeautifulSoup(driver.page_source, 'lxml')
    print(len(soupe.find_all('div', {'data-testid':"property-card"})))
    print(int(''.join(list(filter(str.isdigit, driver.find_element(By.XPATH, "//h1[@aria-live='assertive']").text)))))

load_page_content()

110
131


In [33]:
def get_cards() -> tuple:
    card_count = 0
    cards = []
    try:
        cards = driver.find_elements(By.XPATH, "//div[@data-testid='property-card']")
        card_count = len(cards)
        return cards, card_count 
    except NoSuchElementException:
        return cards, card_count 

In [32]:
        return cards, card_count 

    time.sleep(3)


In [1]:
"dba1b3bddf e99c25fd33 aabf155f9a f42ee7b31a a86bcdb87f b02ceec9d7".replace(" ", ".")

'dba1b3bddf.e99c25fd33.aabf155f9a.f42ee7b31a.a86bcdb87f.b02ceec9d7'

In [34]:
def scroll_down():
    cards, current_card_count = get_cards()
    if cards:
        scroll_to_last_card()
        while True:
            cards, new_card_count = get_cards()
            if new_card_count == current_card_count:
                break
            if new_card_count > current_card_count:
                scroll_to_last_card()


In [38]:
def load_page_content():

    while True:
        try:
            scroll_page()
            btn_vew_more = driver.find_element(By.XPATH, '//*[@id="bodyconstraint-inner"]/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div/button')
            if btn_vew_more:
                driver.execute_script('arguments[0].scrollIntoView({ behavior: "smooth", block: "center", inline: "center" })', btn_vew_more)
                time.sleep(.5)
                btn_vew_more.click()
        except NoSuchElementException:
            break

    soupe = BeautifulSoup(driver.page_source, 'lxml')
    print(len(soupe.find_all('div', {'data-testid':"property-card"})))
    print(int(''.join(list(filter(str.isdigit, driver.find_element(By.XPATH, "//h1[@aria-live='assertive']").text)))))

load_page_content()

90
90
